Use EGG Fourier descriptors
Use the whole EGG segments

Padded
One layer LSTM: no
Try MFCCs

Use EGG Fourier descriptors
Use the whole EGG segments

Padded
One layer LSTM: no
Try MFCCs

Add Window: several inputs predict one output wave form

Reinforce learning: Since the input is not segmented

Giving a fixed length of audio and EGG, choose the representative EGG

不管什么cycles了，直接简单粗暴对应，但是采样低一点？或者EGG可以粗糙一些？或者EGG整体用一个什么东西来表示，或者还是descriptor，但是没有值的时候用零值。

In [ ]:
# From Voice_EGG.wav and Log.aiff generate input and output items.
# Voice_EGG.wav format:
#   Channel 1: Mic signal; Channel 2: EGG signal.
# Log.aiff format:
import wave
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
from google.colab import drive
drive.mount('/content/drive')
from scipy.io import wavfile
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Masking, LSTM, Dense, RepeatVector, TimeDistributed

#drive = r'F:\A2E\test file'
drive = '/content/drive/MyDrive/Colab Notebooks/Test File'

MicFile = ''
CycleFile = 'test_CycleDetection.wav'
FFTDFile = 'test_Log.csv'
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Mounted at /content/drive
Num GPUs Available:  1


In [ ]:
!nvidia-smi

Thu Feb  9 17:44:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
def load_cycle_dataset(folder):
    # Use Cycle file for training
    #
    mic_cycle_suffix = 'test_CycleDetection.wav'
    EGG_suffix = 'test_Log.csv'
    # save Cycle file into two arrays, var cycle contains breakpoints in detecting EGG waveforms
    samplerate, mic_cycle = wavfile.read(os.path.join(drive, 'test_CycleDetection.wav'))
    mic = mic_cycle[:, 0]
    cycle = mic_cycle[:, 1]
    # save log file's first column (time stamp) as np arrays
    with open(os.path.join(folder, EGG_suffix)) as E:
        EGG = np.loadtxt(E, delimiter=',')
        EGG =
    return mic, cycle, EGG
class CycleDataset:
    def __init__(self):
        self.problem_type = 'en-spa'
        self.inp_lang_tokenizer = None
        self.targ_lang_tokenizer = None

    def preprocess(self, audio):
        return audio_processed

SyntaxError: ignored

In [ ]:
# Log file including timing and FFT descriptors
FFTData = os.path.join(drive, FFTDFile)
FFT = np.loadtxt(FFTData, delimiter=',')
sliced = []
for i in range(len(FFT)):
    dit = FFT[i, 0] * 44100
    sliced.append(round(dit))
print(len(sliced))

# Mic data sliced into sequences, --> Input
Samplerate, Mic = wavfile.read(os.path.join(drive, CycleFile))
MicData = Mic[:, 0]
slicedData = []
for i in range(len(sliced)-1):
    temp = MicData[sliced[i]:sliced[i+1]]
    slicedData.append(temp)
max_len = len(slicedData)
max_dim = max([len(d) for d in slicedData])
padded_data = np.zeros((len(slicedData), max_dim))
for i,d in enumerate(slicedData):
  padded_data[i,:len(d)] = d
#slicedData = np.array(slicedData, dtype=object)
#slicedData = np.pad(slicedData,100, 'constant')

# Fourier Arrays --> output
FourierArray = np.array(FFT[:-1, 12:])

# Validation group, used to calculate the distance trained vs. collected
MetricData = FFT[:, [1, 2, 3, 4, 5, 6, 8, 9, 10, 11]]

12425


Pre-Processing

To be functioned

In [ ]:
audio, sr = librosa.load(os.path.join(drive, CycleFile), sr=44100)
# Define the frame size and hop length
frame_size = 1024
hop_length = 512

# Compute the MFCC features
mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20, n_fft=frame_size, hop_length=hop_length)

# Compute the frame duration and average frame rate
frame_duration = frame_size / sr
frame_rate = 1 / (frame_duration * hop_length / sr)

In [ ]:
scaler = MinMaxScaler()

# Fit and transform the data
scaled_FourierArray = scaler.fit_transform(FourierArray)
scaled_padded_data = scaler.fit_transform(padded_data)
# Split data into training and test sets (70% for training and 30% for test)
X_train, X_test, y_train, y_test = train_test_split(scaled_padded_data, scaled_FourierArray, test_size=0.3)

# Split the training data into training and validation sets (70% for training and 30% for validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)



In [ ]:
# Concatenate the input and output data
training_set = np.concatenate((X_train, y_train), axis=1)

# Randomly permute the indices of the training data
permuted_indices = np.random.permutation(training_set.shape[0])
permuted_training_data = training_set[permuted_indices]

# Split the permuted data back into the input and output arrays
X_train, y_train = np.split(permuted_training_data, [X_train.shape[1]], axis=1)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))
#y_train = np.reshape(y_train, (batch_size, y_train.shape[0], y_train.shape[1]))
#y_val = np.reshape(y_val, (batch_size, y_val.shape[0], y_val.shape[1]))



Training session

In [ ]:
inputs = tf.keras.layers.Input(shape=(X_train.shape[1], 1))

# LSTM layer
lstm = tf.keras.layers.LSTM(1024)(inputs)




# Output layer
outputs = tf.keras.layers.Dense(22, activation='softmax')(lstm)

# Model
model = tf.keras.models.Model(inputs, outputs)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Fit the model on the input data
model.fit(X_train[:1000, :, :], y_train[:1000, :], epochs=10, batch_size=8, validation_data=(X_val, y_val))

Epoch 1/10
125/125 [==============================] - 1430s 11s/step - loss: 0.3130 - accuracy: 0.1170 - val_loss: 0.3105 - val_accuracy: 0.0456
Epoch 2/10
125/125 [==============================] - 1428s 11s/step - loss: 0.3122 - accuracy: 0.1270 - val_loss: 0.3105 - val_accuracy: 0.2675
Epoch 3/10
125/125 [==============================] - 1427s 11s/step - loss: 0.3122 - accuracy: 0.1800 - val_loss: 0.3104 - val_accuracy: 0.1821
Epoch 4/10
125/125 [==============================] - 1427s 11s/step - loss: 0.3122 - accuracy: 0.1560 - val_loss: 0.3104 - val_accuracy: 0.1821
Epoch 5/10
125/125 [==============================] - 1427s 11s/step - loss: 0.3121 - accuracy: 0.1610 - val_loss: 0.3104 - val_accuracy: 0.1012
Epoch 6/10
125/125 [==============================] - 1376s 11s/step - loss: 0.3121 - accuracy: 0.1380 - val_loss: 0.3104 - val_accuracy: 0.0517
Epoch 7/10
125/125 [==============================] - 1427s 11s/step - loss: 0.3122 - accuracy: 0.1560 - val_loss: 0.3104 - val_ac

In [ ]:
model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], 1)),
    Dense(22, activation='softmax')
])
model.summary()
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64, verbose=2)


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)

    def forward(self, input, hidden, cell):
        output, (hidden, cell) = self.lstm(input, (hidden, cell))
        return output, hidden, cell

    def init_hidden_cell(self, batch_size):
        hidden = torch.zeros(1, batch_size, self.hidden_size)
        cell = torch.zeros(1, batch_size, self.hidden_size)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(hidden_size, output_size)

    def forward(self, input, hidden, cell):
        output, (hidden, cell) = self.lstm(input, (hidden, cell))
        return output, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_forcing_ratio = 0.5):
        batch_size = source.size(1)
        target_len = target.size(0)
        target_vocab_size = self.decoder.lstm.out_features

        outputs = torch.zeros(target_len, batch_size, target_vocab_size)

        hidden, cell = self.encoder.init_hidden_cell(batch_size)
        encoder_output, hidden, cell = self.encoder(source, hidden, cell)

        decoder_input = torch.zeros(1, batch_size, target_vocab_size)

        for t in range(target_len):
            decoder_output, hidden, cell = self.decoder(decoder_input, hidden, cell)
            outputs[t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio
            decoder_input = target[t] if teacher_force else decoder_output.argmax(1).unsqueeze(0)

        return outputs

In [ ]:
encoder = Encoder()
model = Seq2Seq()
model.forward(X_train, y_train)

Test

In [ ]:
# Evaluate the model on test data
score = model.evaluate(X_test, y_test, verbose=2)